In [34]:
import pandas as pd
import numpy as np
import re

## ✂️ extract loai-bds and quan


### extract data from link

In [35]:
df = pd.read_csv('raw.csv')

pattern = r'(?<=\.vn)/([^/]+/[^/]+/)'
df['data'] = df['link'].apply(lambda x: re.search(pattern, x).group(1) if re.search(pattern, x) else '')
print(df.head())


                                                link            price    area  \
0  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...             3 tỷ   62 m2   
1  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...   4 tỷ 500 triệu  105 m2   
2  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...   3 tỷ 700 triệu   83 m2   
3  https://mogi.vn/quan-ngu-hanh-son/mua-nha-biet...  12 tỷ 500 triệu  427 m2   
4  https://mogi.vn/quan-son-tra/mua-dat-tho-cu/ap...  13 tỷ 600 triệu  210 m2   

   room toilet                                            address       lat  \
0  1 PN   1 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
1  3 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
2  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
3  3 PN   3 WC  Trường Sa, Phường Hòa Hải, Quận Ngũ Hành Sơn, ...  15.99582   
4  0 PN   0 WC  Ngô Quyền, Phường Thọ Quang, Quận Sơn Trà, Đà ...  16.09384   

        long                          

### split quan and loai-bds from data

In [36]:
# Split the "data" column into two columns based on the slash separator
df[['quan', 'loai_bds']] = df['data'].str.split('/', n=1, expand=True)

# Drop the original "data" column
df.drop('data', axis=1, inplace=True)
print(df.head())

                                                link            price    area  \
0  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...             3 tỷ   62 m2   
1  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...   4 tỷ 500 triệu  105 m2   
2  https://mogi.vn/quan-ngu-hanh-son/mua-can-ho-c...   3 tỷ 700 triệu   83 m2   
3  https://mogi.vn/quan-ngu-hanh-son/mua-nha-biet...  12 tỷ 500 triệu  427 m2   
4  https://mogi.vn/quan-son-tra/mua-dat-tho-cu/ap...  13 tỷ 600 triệu  210 m2   

   room toilet                                            address       lat  \
0  1 PN   1 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
1  3 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
2  2 PN   2 WC  Phạm Kiệt, Phường Khuê Mỹ, Quận Ngũ Hành Sơn, ...  16.03851   
3  3 PN   3 WC  Trường Sa, Phường Hòa Hải, Quận Ngũ Hành Sơn, ...  15.99582   
4  0 PN   0 WC  Ngô Quyền, Phường Thọ Quang, Quận Sơn Trà, Đà ...  16.09384   

        long               quan       

## 🧹 ['price'] - ['room'] -['toilet']

In [37]:
def convert_to_dong(string):
    if not isinstance(string, str):
        return 0
    pattern = r'(\d+)\s*(tỷ)?\s*(\d+)?\s*(triệu)?'
    string = string.replace('.', '')
    match = re.search(pattern, string)
    
    if match.group(2) and match.group(4):
        return int(match.group(1)) * 1000000000 + int(match.group(3)) * 1000000
    elif match.group(2):
        return int(match.group(1)) * 1000000000
    elif match.group(4):
        return int(match.group(1)) * 1000000
    else:
        return int(match.group(1))

def convert_to_number(string):
    numbers = re.findall(r'\d+', string)
    return numbers[0]

### 👀 Check missing values and test function convert

In [38]:
print(df.isna().sum())
print(convert_to_dong('5 tỷ'))
print(convert_to_number('210 m2'))

link        0
price       2
area        0
room        0
toilet      0
address     0
lat         0
long        0
quan        0
loai_bds    0
dtype: int64
5000000000


In [39]:

df['price'].fillna(0)
df['price-VND'] = df['price'].apply(convert_to_dong)
df = df.sort_values('price-VND', ascending=True)
df['room']=df['room'].apply(convert_to_number).astype('Int64')
df['toilet']=df['toilet'].apply(convert_to_number).astype('Int64')
df['area']=df['area'].apply(convert_to_number).astype('Int64')


In [40]:
df

,link,price,area,room,toilet,address,lat,long,quan,loai_bds,price-VND
7164,https://mogi.vn/quan-lien-chieu/mua-dat-nen-du...,NaN,100,0,0,"Nguyễn Lương Bằng, Phường Hoà Hiệp Nam, Quận L...",16.09512,108.13844,quan-lien-chieu,mua-dat-nen-du-an/,0
5069,https://mogi.vn/quan-lien-chieu/mua-dat-nen-du...,NaN,100,0,0,"Nguyễn Tất Thành, Phường Hoà Hiệp Nam, Quận Li...",16.10610,108.13558,quan-lien-chieu,mua-dat-nen-du-an/,0
404,https://mogi.vn/quan-hai-chau/mua-nha-mat-tien...,100.000 đ,120,4,5,"Bạch Đằng, Phường Hải Châu I, Quận Hải Châu, Đ...",16.07079,108.22511,quan-hai-chau,mua-nha-mat-tien-pho/,100000
405,https://mogi.vn/quan-hai-chau/mua-nha-biet-thu...,100.000 đ,120,4,6,"Hoá Sơn 10, Phường Hòa Cường Nam, Quận Hải Châ...",16.03346,108.23031,quan-hai-chau,mua-nha-biet-thu-lien-ke/,100000
4303,https://mogi.vn/quan-son-tra/mua-nha-mat-tien-...,105.000 đ,81,7,7,"Thạch Lam, Phường Phước Mỹ, Quận Sơn Trà, Đà Nẵng",16.05685,108.24449,quan-son-tra,mua-nha-mat-tien-pho/,105000
...,...,...,...,...,...,...,...,...,...,...,...
4279,https://mogi.vn/quan-hai-chau/mua-nha-mat-tien...,5500 tỷ,76,3,2,"Nguyễn Thi, Phường Hòa Cường Nam, Quận Hải Châ...",16.03214,108.23095,quan-hai-chau,mua-nha-mat-tien-pho/,5500000000000
2133,https://mogi.vn/quan-cam-le/mua-nha-mat-tien-p...,6992 tỷ,92,3,2,"Tú Mỡ, Phường Hòa An, Quận Cẩm Lệ, Đà Nẵng",16.05304,108.17310,quan-cam-le,mua-nha-mat-tien-pho/,6992000000000
2120,https://mogi.vn/quan-thanh-khe/mua-nha-mat-tie...,7800 tỷ,100,4,2,"Thanh Huy 1, Phường Thanh Khê Đông, Quận Thanh...",16.06942,108.18316,quan-thanh-khe,mua-nha-mat-tien-pho/,7800000000000
2111,https://mogi.vn/quan-ngu-hanh-son/mua-nha-mat-...,14798 tỷ,151,3,2,"Nguyễn Văn Thoại, Phường Mỹ An, Quận Ngũ Hành ...",16.05638,108.24681,quan-ngu-hanh-son,mua-nha-mat-tien-pho/,14798000000000


## ✂️ Drop the unnecessary columns


In [41]:
df.drop(['link','price','address'],inplace=True, axis=1)
df

,area,room,toilet,lat,long,quan,loai_bds,price-VND
7164,100,0,0,16.09512,108.13844,quan-lien-chieu,mua-dat-nen-du-an/,0
5069,100,0,0,16.10610,108.13558,quan-lien-chieu,mua-dat-nen-du-an/,0
404,120,4,5,16.07079,108.22511,quan-hai-chau,mua-nha-mat-tien-pho/,100000
405,120,4,6,16.03346,108.23031,quan-hai-chau,mua-nha-biet-thu-lien-ke/,100000
4303,81,7,7,16.05685,108.24449,quan-son-tra,mua-nha-mat-tien-pho/,105000
...,...,...,...,...,...,...,...,...
4279,76,3,2,16.03214,108.23095,quan-hai-chau,mua-nha-mat-tien-pho/,5500000000000
2133,92,3,2,16.05304,108.17310,quan-cam-le,mua-nha-mat-tien-pho/,6992000000000
2120,100,4,2,16.06942,108.18316,quan-thanh-khe,mua-nha-mat-tien-pho/,7800000000000
2111,151,3,2,16.05638,108.24681,quan-ngu-hanh-son,mua-nha-mat-tien-pho/,14798000000000


## 📝 Export data to csv

In [42]:
df.to_csv('clean.csv',index=False)